In [1]:
import numpy as np
from torch.utils.data import DataLoader
import pandas as pd
import os
import torch.utils.data as data
import math
import ast
import cv2
import matplotlib as plt
from PIL import Image



x = np.load(r"D:\\School\\Lab\\Compact-Gesture-Transformer-Code\\rgb_train.npz", allow_pickle=True)['arr_0']
print(type(x))

<class 'numpy.ndarray'>


In [2]:
list = x.tolist()
x[0]

{'session': 25,
 'record': 0,
 'label': 0,
 'data': ['rgb/train/025/g00/00/rgb/000_rgb.png',
  'rgb/train/025/g00/00/rgb/001_rgb.png',
  'rgb/train/025/g00/00/rgb/002_rgb.png',
  'rgb/train/025/g00/00/rgb/003_rgb.png',
  'rgb/train/025/g00/00/rgb/004_rgb.png',
  'rgb/train/025/g00/00/rgb/005_rgb.png',
  'rgb/train/025/g00/00/rgb/006_rgb.png',
  'rgb/train/025/g00/00/rgb/007_rgb.png',
  'rgb/train/025/g00/00/rgb/008_rgb.png',
  'rgb/train/025/g00/00/rgb/009_rgb.png',
  'rgb/train/025/g00/00/rgb/010_rgb.png',
  'rgb/train/025/g00/00/rgb/011_rgb.png',
  'rgb/train/025/g00/00/rgb/012_rgb.png',
  'rgb/train/025/g00/00/rgb/013_rgb.png',
  'rgb/train/025/g00/00/rgb/014_rgb.png',
  'rgb/train/025/g00/00/rgb/015_rgb.png',
  'rgb/train/025/g00/00/rgb/016_rgb.png',
  'rgb/train/025/g00/00/rgb/017_rgb.png',
  'rgb/train/025/g00/00/rgb/018_rgb.png',
  'rgb/train/025/g00/00/rgb/019_rgb.png',
  'rgb/train/025/g00/00/rgb/020_rgb.png',
  'rgb/train/025/g00/00/rgb/021_rgb.png',
  'rgb/train/025/g00/00/r

In [3]:
from torch.utils.data.dataset import Dataset
import torch
class Briareo_csv(Dataset):
    def __init__(self, images_path, transform=None):
        self.transform = transform
        self.image_path = np.load(images_path, allow_pickle=True)['arr_0']
        self.image_path.tolist()
        
    def __len__(self):
        return len(self.image_path)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return self.image_path[idx]['label'], self.image_path[idx]['data']

In [4]:
def generate_csv(root, split, path):
    dic_target_img_dir = {}
    for index, (id, path) in enumerate(DataLoader(path)):
        dic_target_img_dir[index] = {'class_id': id.item(), 'dir': path}
        
    df = pd.DataFrame.from_dict(dic_target_img_dir, orient='index')
    fp = os.path.join(root, f"{split}.csv")
    df.to_csv(fp, header = True, index= False)
        

In [ ]:
split_root = r"D:\School\Lab\Compact-Gesture-Transformer-Code\data\Briareo\rgb"
split = 'train'
train_path = Briareo_csv("data\\Briareo\\rgb\\splits\\train\\rgb_train.npz", transform=None)
generate_csv(root = split_root, split = split, path = train_path)


In [6]:
split_root = r"D:\School\Lab\Compact-Gesture-Transformer-Code\data\Briareo\rgb"
split = 'test'
test_path = Briareo_csv("data\\Briareo\\rgb\\splits\\test\\rgb_test.npz", transform=None)
generate_csv(root = split_root, split = split, path = test_path)

In [7]:
class DatasetImgTarget(data.Dataset):
    def __init__(self, root, df_train, split, transform = None , n_frames = 8):
        self.transform = transform
        self.root = root
        self.split = split
        self.images_folder = root
        self.n_frames = n_frames
        
        if(split=='train'):
            self.df_file_name = df_train
        
        path_csv = os.path.join(root,'rgb', f'{split}.csv')
        self.df = pd.read_csv(path_csv, sep=',')
        self.targets = self.df['class_id'].to_numpy()
        self.data = self.df['dir'].to_numpy()
        
        # Stores the path to images selected after sampling
        fixed_data = []
        
        for i, record in enumerate(self.data):
            # used ast because the array being passed record is in the form of a string
            # For more info, try printing type(record) or print len(record)
            record = ast.literal_eval(record)
            center_of_list = math.floor(len(record)/2)
            crop_limit = math.floor(self.n_frames / 2)
            start = center_of_list - crop_limit
            end = center_of_list + crop_limit 
            # Add one more extra frame if n_frames is odd  
            paths_cropped = record[start: end + 1 if self.n_frames % 2 == 1 else end]
            # Adding arrays of cropped clips for every video_sample
            fixed_data.append(paths_cropped)
            
        self.data = fixed_data
                    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        paths = self.data[index]
        label = self.targets[index]
        
        clip = []
        
        # Process each frame in a gesture clip
        for p in paths:
            # p[0] because apparently p is a tuple
            path_frame = os.path.join(self.root, p[0])
            print(path_frame)
            img = cv2.imread(path_frame)
            # cv2.imshow("image", img)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            img = cv2.resize(img, (224, 224))
            clip.append(img)
            
        # Clip looked like this (224, 224, 3, 8) 
        # Convert array to a numpy array
        clip = np.array(clip)
        # the -1 at the end automatically computes the last dimension (infer?)
        # Becomes (24, 224, 224) which represents channel, height, width
        clip = torch.from_numpy(clip.reshape(clip.shape[1], clip.shape[2], -1).transpose(2, 0, 1))
        label = torch.LongTensor(np.asarray([label]))
        
        
        
        return clip.float(), label

In [8]:
train_set = DatasetImgTarget(root=r"data\\Briareo", split='train', df_train='train')
for index, data in enumerate(DataLoader(train_set)):
    print(index)
    print(data)

data\\Briareo\rgb/train/025/g00/00/rgb/020_rgb.png
data\\Briareo\rgb/train/025/g00/00/rgb/021_rgb.png
data\\Briareo\rgb/train/025/g00/00/rgb/022_rgb.png
data\\Briareo\rgb/train/025/g00/00/rgb/023_rgb.png
data\\Briareo\rgb/train/025/g00/00/rgb/024_rgb.png
data\\Briareo\rgb/train/025/g00/00/rgb/025_rgb.png
data\\Briareo\rgb/train/025/g00/00/rgb/026_rgb.png
data\\Briareo\rgb/train/025/g00/00/rgb/027_rgb.png
0
[tensor([[[[  0.,   0.,   0.,  ..., 199., 171., 166.],
          [  0.,   0.,   8.,  ..., 190., 176., 163.],
          [  0.,  25.,  22.,  ..., 188., 173., 164.],
          ...,
          [  0.,   0.,   0.,  ..., 216., 195., 179.],
          [  0.,   0.,   0.,  ..., 215., 191., 180.],
          [  0.,   0.,   0.,  ..., 207., 190., 168.]],

         [[  5.,   3.,   2.,  ..., 212., 192., 181.],
          [  4.,   5.,   8.,  ..., 203., 190., 182.],
          [  6.,  22.,  40.,  ..., 195., 188., 177.],
          ...,
          [  1.,   2.,   7.,  ..., 232., 212., 189.],
          [  1., 

KeyboardInterrupt: 